In [10]:
!pip install requests
!pip install beautifulsoup4
!pip install pandas

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to extract data from a single page
def scrape_page_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    movies = soup.find_all('div', class_='lister-item-content')

    data_csv1 = []
    data_csv2 = []

    for movie in movies:
        movie_name = movie.h3.a.text
        directors = movie.find_all('p')[2].find_all('a')
        directors_list = [director.text for director in directors]
        duration = movie.find('span', class_='runtime').text
        year = movie.find('span', class_='lister-item-year').text.strip('()')
        rating = movie.strong.text
        metascore = movie.find('span', class_='metascore').text.strip() if movie.find('span', class_='metascore') else ''
        stars = movie.find('p', class_='').find_all('a')
        stars_list = [star.text for star in stars]
        votes = movie.find('span', attrs={'name': 'nv'}).text
        genre = movie.find('span', class_='genre').text.strip()
        gross = movie.find_all('span', attrs={'name': 'nv'})[-1].text if len(movie.find_all('span', attrs={'name': 'nv'})) > 1 else ''
        popularity = movie.find('span', class_='lister-item-index unbold text-primary').text
        certification = movie.find('span', class_='certificate').text if movie.find('span', class_='certificate') else ''

        data_csv1.append([movie_name, directors_list, duration, year, rating, metascore])
        data_csv2.append([movie_name, stars_list, votes, genre, gross, popularity, certification])

    return data_csv1, data_csv2

# Function to store data in CSV files
def create_csv(data, columns, filename):
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(filename, index=False)

# IMDb URL for pagination
base_url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1&page="

# Scrape data from all pages
data_csv1_all = []
data_csv2_all = []

current_page = 1
while current_page<=36 :
    url = base_url + str(current_page)
    print(f"Scraping data from page {current_page}...")
    data_csv1, data_csv2 = scrape_page_data(url)

    if not data_csv1 or not data_csv2:
        break

    data_csv1_all.extend(data_csv1)
    data_csv2_all.extend(data_csv2)

    current_page += 1

# CSV 1 columns
csv1_columns = ['Movie Name', 'Director1', 'Director2', 'Duration', 'Year', 'Ratings', 'Metascore']

# CSV 2 columns
csv2_columns = ['Movie Name', 'Star1', 'Star2', 'Star3', 'Star4', 'Votes', 'Genre1', 'Genre2', 'Genre3', 'Gross collection', 'Popularity', 'Certification']

# Splitting data for CSV 1
csv1_data = []
for item in data_csv1_all:
    movie_name, directors, duration, year, rating, metascore = item
    director1 = directors[0] if len(directors) > 0 else ''
    director2 = directors[1] if len(directors) > 1 else ''
    csv1_data.append([movie_name, director1, director2, duration, year, rating, metascore])

# Splitting data for CSV 2
csv2_data = []
for item in data_csv2_all:
    movie_name, stars, votes, genre, gross, popularity, certification = item
    star1 = stars[0] if len(stars) > 0 else ''
    star2 = stars[1] if len(stars) > 1 else ''
    star3 = stars[2] if len(stars) > 2 else ''
    star4 = stars[3] if len(stars) > 3 else ''
    genres = genre.split(',')
    genre1 = genres[0].strip() if len(genres) > 0 else ''
    genre2 = genres[1].strip() if len(genres) > 1 else ''
    genre3 = genres[2].strip() if len(genres) > 2 else ''
    csv2_data.append([movie_name, star1, star2, star3, star4, votes, genre1, genre2, genre3, gross, popularity, certification])

# Creating CSV files
create_csv(csv1_data, csv1_columns, 'imdb_data_csv1_all.csv')
create_csv(csv2_data, csv2_columns, 'imdb_data_csv2_all.csv')

print("Data has been scraped from all pages and saved to CSV files successfully.")


Scraping data from page 1...
Scraping data from page 2...
Scraping data from page 3...
Scraping data from page 4...
Scraping data from page 5...
Scraping data from page 6...
Scraping data from page 7...
Scraping data from page 8...
Scraping data from page 9...
Scraping data from page 10...
Scraping data from page 11...
Scraping data from page 12...
Scraping data from page 13...
Scraping data from page 14...
Scraping data from page 15...
Scraping data from page 16...
Scraping data from page 17...
Scraping data from page 18...
Scraping data from page 19...
Scraping data from page 20...
Scraping data from page 21...
Scraping data from page 22...
Scraping data from page 23...
Scraping data from page 24...
Scraping data from page 25...
Scraping data from page 26...
Scraping data from page 27...
Scraping data from page 28...
Scraping data from page 29...
Scraping data from page 30...
Scraping data from page 31...
Scraping data from page 32...
Scraping data from page 33...
Scraping data from 

In [16]:
import sqlite3
import pandas as pd

# Function to create the SQLite database, tables, and insert data
def create_database_and_tables():
    conn = sqlite3.connect('imdb_data.db')
    cursor = conn.cursor()

    # Create Table 1
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Table1 (
        Sno INTEGER PRIMARY KEY,
        MovieName TEXT,
        DirectorName TEXT,
        Duration TEXT,
        Genre TEXT,
        Ratings REAL
    )
    ''')

    # Create Table 2
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Table2 (
        MovieName TEXT PRIMARY KEY,
        Stars TEXT,
        Votes INTEGER,
        Genre TEXT,
        GrossCollection TEXT,
        Popularity TEXT,
        Certification TEXT
    )
    ''')

    # Load CSV data into data frames
    df1 = pd.read_csv('imdb_data_csv1_all.csv')
    df2 = pd.read_csv('imdb_data_csv2_all.csv')

    # Insert data into Table 1
    df1.to_sql('Table1', conn, if_exists='replace', index=False)

    # Insert data into Table 2
    df2.to_sql('Table2', conn, if_exists='replace', index=False)
    
    print(df1)
    print(df2)
    # Commit and close the connection
    conn.commit()
    conn.close()

# Create the database, tables, and insert data
create_database_and_tables()



                                             Movie Name           Director1  \
0                                       The Dark Knight   Christopher Nolan   
1         The Lord of the Rings: The Return of the King       Peter Jackson   
2                   Spider-Man: Across the Spider-Verse  Joaquim Dos Santos   
3                                             Inception   Christopher Nolan   
4     The Lord of the Rings: The Fellowship of the Ring       Peter Jackson   
...                                                 ...                 ...   
1795                 Kimetsu no Yaiba: Mugen Ressha-Hen      Haruo Sotozaki   
1796                        Baahubali 2: The Conclusion      S.S. Rajamouli   
1797                                 Gangs of Wasseypur      Anurag Kashyap   
1798                                            Yôjinbô      Akira Kurosawa   
1799                           Uri: The Surgical Strike         Aditya Dhar   

              Director2 Duration  Year  Ratings  Me